In [4]:
from services.Coinmarketcap import CoinMarketCap
from dotenv import dotenv_values
import pprint
import json

In [5]:
config = dotenv_values('.env')
cmc = CoinMarketCap(config.get("MARKET_CAP_AP_KEY"))

In [13]:
response = cmc.get_coin_price(coin='BTC')

In [20]:
pprint.pprint(json.loads(response)['data']['BTC'][0]['quote']['USD']['price'])

65800.38146848558


In [1]:
from services.Database import Database

In [2]:
database = Database(worksheets=[
    ("DATA", 10),
    ("COINS", 2),
    ("EXCHANGES", 1)
])

2024-06-06 08:48:32.330 
  command:

    streamlit run /home/guilherme/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-06-06 08:48:32.372 No runtime found, using MemoryCacheStorageManager
2024-06-06 08:48:32.374 No runtime found, using MemoryCacheStorageManager
2024-06-06 08:48:34.276 No runtime found, using MemoryCacheStorageManager
2024-06-06 08:48:35.266 No runtime found, using MemoryCacheStorageManager


In [3]:
register_sheet = database.worksheets["DATA"].dropna(how="all")
coin_sheet = database.worksheets["COINS"].dropna(how="all")
exchange_sheet = database.worksheets["EXCHANGES"].dropna(how="all")

In [5]:
exchange_filtered = register_sheet[register_sheet['Exchange'] == 'Binance']

In [19]:
coin_current_price = exchange_filtered[['Coin', 'Preço Atual (R$)']].drop_duplicates('Coin', keep='first')
coin_current_price = {coin:price for coin, price in zip(coin_current_price['Coin'], coin_current_price['Preço Atual (R$)'])}

In [20]:
coin_qte = {}

In [6]:
exchange_filtered.head()

,Data,Coin,Preço (R$),Preço (U$),Preço Atual (R$),Preço Atual (U$),Valor Investido (R$),Qte,Status,Exchange
0,19/04/24,BTC,334250.0,66850.00,363457.044599,69895.585500,1000.0,0.002992,Buy,Binance
1,19/04/24,ETH,15760.0,3152.00,19686.267078,3785.820592,500.0,0.031726,Buy,Binance
2,19/04/24,MATIC,4.6,0.92,3.696202,0.710808,1500.0,326.086957,Buy,Binance
3,19/04/24,SOL,677.1,135.42,865.051094,166.355980,250.0,0.369222,Buy,Binance
4,19/04/24,LINK,66.7,13.34,92.380350,17.765452,250.0,3.748126,Buy,Binance


In [7]:
buy_rebuy_df = exchange_filtered[(exchange_filtered['Status'] == 'Buy') | (exchange_filtered['Status'] == 'Rebuy')]

In [25]:
buy_rebuy_df = buy_rebuy_df.groupby('Coin')['Qte'].sum().reset_index()

,Coin,Qte
0,AEVO,1.126571e+02
1,BTC,2.991773e-03
2,ETH,3.172589e-02
3,IMX,2.857143e+01
4,LINK,3.748126e+00
5,MATIC,3.260870e+02
6,PEPE,5.022831e+07
7,SHIB,6.383320e+07
8,SOL,3.692217e-01
9,VET,2.196195e+03


In [26]:
sell_df = exchange_filtered[exchange_filtered['Status'] == 'Sell']
sell_df = sell_df.groupby('Coin')['Qte'].sum().reset_index()
sell_df

,Coin,Qte
0,PEPE,4.165021e+07


In [34]:
for coin in coin_current_price.keys():
    qte = 0.0
    if coin in buy_rebuy_df['Coin'].values:
        qte_buy = buy_rebuy_df.loc[buy_rebuy_df['Coin'] == coin]['Qte'].values[0]
        qte += qte_buy
    
    if coin in sell_df['Coin'].values:
        qte_sell = sell_df.loc[sell_df['Coin'] == coin]['Qte'].values[0]
        qte -= qte_sell

    coin_current_price[coin] = coin_current_price[coin] * qte

In [35]:
coin_current_price

{'BTC': 1087.3808364955098,
 'ETH': 624.564310840925,
 'MATIC': 1205.2832444271833,
 'SOL': 319.3956188776135,
 'LINK': 346.253187582264,
 'IMX': 338.49856287962905,
 'SHIB': 7987.007930117598,
 'PEPE': 643.4512877569438,
 'AEVO': 557.021238785228,
 'VET': 388.1981760046969}

In [42]:
sum(coin_current_price.values())

13497.05439376759

In [47]:
import pandas as pd

In [53]:
exchange_filtered['Data'] = pd.to_datetime(exchange_filtered['Data'])
rebase_df = exchange_filtered[8:]

In [56]:
exchange_filtered[:8]['Valor Investido (R$)'].sum()

9380.0